In [ ]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [21]:
mongo = MongoClient(port=27017)

In [22]:
db = mongo['uk_food']

In [ ]:
print(db.list_collection_names())

In [24]:
establishments = db['establishments']

In [ ]:
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"There are {count} establishments with a hygiene score of 20.")
      
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(f"There are {len(df)} rows in this DataFrame.")

# Display the first 10 rows of the DataFrame
df.head(10)

In [ ]:
query2 = {'LocalAuthorityName': {'$regex': 'London'},
         'RatingValue': {'$gte': '4'}
         }

# Use count_documents to display the number of documents in the result
count2 = establishments.count_documents(query2)
print(f"There are {count2} establishments in London that have a RatingValue greater than or equal to 4.")

# Display the first document in the results using pprint
pprint(establishments.find_one(query2))

In [ ]:
df2 = pd.DataFrame(establishments.find(query2))

# Display the number of rows in the DataFrame
print(f"There are {len(df2)} rows in this DataFrame.")

# Display the first 10 rows of the DataFrame
df.head(10)

In [ ]:
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query3 = {'RatingValue': '5',
         'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
         'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)} 
         }
sort =  [('scores.Hygiene', 1)]
limit = 5

# Print the results
for result in establishments.find(query3).sort(sort).limit(limit):
    pprint(result)

In [ ]:
pd.DataFrame(establishments.find(query3).sort(sort).limit(limit))

In [ ]:
query_match = {'$match': {'scores.Hygiene': 0}}

query_group = {'$group': {'_id': '$LocalAuthorityName',
                         'count': {'$sum':1}
                         }
              }
query_sort = {'$sort': {'count': -1}}

pipeline = [query_match, query_group, query_sort]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results)} documents in the result.")

# Print the first 10 results
pprint(results[0:10])

In [ ]:
pipe_df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"There are {len(pipe_df)} rows in the DataFrame.")

# Display the first 10 rows of the DataFrame
pipe_df.head(10